# **A Novel Explainable Approach for Intrusion Detection**

This notebook presents the code for the the paper **A Novel Explainable Approach for Intrusion Detection**.  


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Reading the dataset

To access the **CIC-IDS 2017** dataset, follow these steps:

1. Visit the official dataset page: [CIC-IDS 2017](https://www.unb.ca/cic/datasets/ids-2017.html).
2. Scroll to the bottom of the page and click on **"Download this dataset"**.
3. Fill in the required details, including your **name, email address, and other requested information**.
4. Click on the **Submit** button.
5. Once submitted, a page will appear containing a folder named **CIC-IDS-2017**.
6. Navigate to the **CSVs** folder inside this directory.
7. Download the file **MachineLearningCSV.zip** and extract its contents.

After extraction, you will find the following CSV files:

- Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv 
- Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv 
- Friday-WorkingHours-Morning.pcap_ISCX.csv      
- Monday-WorkingHours.pcap_ISCX.csv            
- Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv 
- Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv 
- Tuesday-WorkingHours.pcap_ISCX.csv           
- Wednesday-workingHours.pcap_ISCX.csv          


For this research, we selected the file **`Wednesday-workingHours.pcap_ISCX.csv`** which contains **692,703 records** each consists of **68 attributes (features)** 

In [2]:
# Set Pandas to display all rows without truncation (useful for large datasets)
pd.set_option('display.max_row', None)

# Set Pandas to display all columns without truncation (ensures all features are visible)
pd.set_option('display.max_columns', None)

# Read the CSV file into a Pandas DataFrame (loading network traffic data for analysis)
# The file 'Wednesday-workingHours.pcap_ISCX.csv' must be in the same directory as the 
# 'main.ipynb'. Otherwise, you need to provide the full file path.

df = pd.read_csv('Wednesday-workingHours.pcap_ISCX.csv')  # df: the original dataset

# Pre-processing

The preprocessing consists of the following steps:  

### **1. Handling Missing and Out-of-Range Values**  
- Samples with missing or out-of-range values are **identified and removed** instead of being imputed.  
- Imputation is avoided to maintain data integrity, as the number of affected samples is negligible.  

### **2. Removing Zero-Variance Features**  
- Features with **zero variance** (i.e., identical values across all records) are eliminated.  
- These features do not contribute to model learning and can slow down computations.  
- **Examples of zero-variance features:**  
  - `Bwd PSH Flags`, `Fwd URG Flags`, `Bwd URG Flags`, `CWE Flag Count`, etc.  

### **3. Eliminating Duplicate Features**  
- Identical features providing redundant information are detected and removed.  
- Retaining only one feature per duplicate set reduces computational overhead and improves model interpretability. 🚀  

   


In [3]:
# data Cleaning (df_cleaned)
df_cleaned = df.dropna()  # Drop rows with NaN values
df_cleaned = df_cleaned[~df_cleaned.isin([np.inf, -np.inf]).any(axis=1)] # Drop rows with out of range values

In [4]:
# Seprating labels and data
data = df_cleaned.iloc[:,:-1]   # data contains clean attributes
labels = df_cleaned.iloc[:,-1]  # labels contain the corresponding labels in categorical format ('BENIGN','DoS slowloris',... )

In [5]:
# identifying zero-variance features
zero_variance_features = data.var() == 0
zero_variance_index = zero_variance_features[zero_variance_features].index
print("Features with zero variance:\n", zero_variance_index)

Features with zero variance:
 Index([' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' CWE Flag Count',
       'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate',
       ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')


In [6]:
# Droping the specified columns (i.e., those with zero variance)
data_dropped_zero_var = data.drop(columns= zero_variance_index)

In [7]:
# Finding identical columns
columns = data_dropped_zero_var.columns
identical_columns = []

for col1 in data_dropped_zero_var.columns:
    for col2 in data_dropped_zero_var.columns:
        if col1 != col2 and data_dropped_zero_var[col1].equals(data_dropped_zero_var[col2]):
            identical_columns.append((col1, col2))

print("Identical columns:", identical_columns)

Identical columns: [(' Total Fwd Packets', 'Subflow Fwd Packets'), (' Total Backward Packets', ' Subflow Bwd Packets'), ('Total Length of Fwd Packets', ' Subflow Fwd Bytes'), (' Fwd Packet Length Mean', ' Avg Fwd Segment Size'), ('Fwd PSH Flags', ' SYN Flag Count'), (' Fwd Header Length', ' Fwd Header Length.1'), (' SYN Flag Count', 'Fwd PSH Flags'), (' Avg Fwd Segment Size', ' Fwd Packet Length Mean'), (' Fwd Header Length.1', ' Fwd Header Length'), ('Subflow Fwd Packets', ' Total Fwd Packets'), (' Subflow Fwd Bytes', 'Total Length of Fwd Packets'), (' Subflow Bwd Packets', ' Total Backward Packets')]


In [8]:

# Use a set to filter out tuples considering (X, Y) and (Y, X) as the same
unique_pairs = set()

# Add each tuple to the set in a sorted order (to avoid duplicates with reversed order)
for pair in identical_columns:
    unique_pairs.add(tuple(sorted(pair)))

# Convert back to list to maintain the same structure as original input
unique_tuples_list = list(unique_pairs)

# Output the result
print(unique_tuples_list)

[(' Subflow Bwd Packets', ' Total Backward Packets'), (' SYN Flag Count', 'Fwd PSH Flags'), (' Total Fwd Packets', 'Subflow Fwd Packets'), (' Fwd Header Length', ' Fwd Header Length.1'), (' Subflow Fwd Bytes', 'Total Length of Fwd Packets'), (' Avg Fwd Segment Size', ' Fwd Packet Length Mean')]


In [9]:
# Extract the columns to drop (second item of each tuple)
columns_to_drop = [pair[1] for pair in unique_tuples_list]

In [10]:
# Drop the duplicate columns from the DataFrame
data_var_ident_dropped = data_dropped_zero_var.drop(columns=columns_to_drop)

# Covariance Matrix of Preprocessed Data

After completing the preprocessing steps, we calculate the covariance matrix of the remaining features. Before this calculation, feature scaling is applied by standardizing the features. The resulting covariance matrix has several key properties:  

- **Diagonal Values**: All diagonal values are large, indicating that features with zero variance have been removed during preprocessing.  
- **Symmetry**: The covariance matrix is symmetric, as expected.  
- **Feature Covariance**: The matrix reveals strong covariance among certain features, suggesting a high degree of correlation. 

In [11]:

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_var_ident_dropped)  # it converts pandas dataframe into a numpy array
##########################################################
# After applying StandardScaler to data_var_ident_dropped, 
# the resulting scaled_data is a NumPy array without any
# column names. To retrieve the corresponding column names,
# you can use the column names from the original DataFrame,
# 'data_var_ident_dropped'.
##########################################################


# Compute the Covariance Matrix (for the standardized data)
cov_matrix = np.cov(scaled_data, rowvar=False) 

In [12]:
%matplotlib qt
# Set the plot size
plt.figure(figsize=(10, 8))

# Create the heatmap using seaborn
sns.heatmap(cov_matrix, annot=False, cmap="coolwarm", fmt='.2f', linewidths=0.5)

# Add labels and title
# plt.title('Covariance Matrix Heatmap', fontsize=12)
plt.xlabel('Features', fontsize=12)
plt.ylabel('Features', fontsize=12)

# Show the plot
plt.show()

# Changing the labels

In the dataset, the class *BENIGN* represents normal activities, while the other five classes correspond to different types of malicious attacks. To align with an anomaly detection framework, the labels are modified as follows:
- **Label 0** is assigned to **BENIGN** samples (*normal activities*).
- **Label 1** is assigned to all other samples (*malicious attacks*).

In [13]:
# Convert categorical labels to numerical values
labels_numeric = labels.apply(lambda x: 0 if x == 'BENIGN' else 1)

# Combine Cleaned Data with Corresponding Labels

In [14]:
# Concatenate labels with cleaned data
cleaned_data_labels = pd.concat([data_var_ident_dropped, labels_numeric.rename('Label')], axis=1)

# Define Features and Target Variable

In [15]:
# Define features and target variable
X = cleaned_data_labels.iloc[:, :-1]  # features: all columns except the last one 
y = cleaned_data_labels.iloc[:, -1]   # targets : last column 

# Training XGBoost and extracting importance scores

### Overview
Understanding which features contribute most to model predictions is crucial for improving performance and interpretability. XGBoost provides multiple ways to measure feature importance, helping us identify the most influential features in the dataset. 

In this section, we:
- Train an **XGBoost model** on preprocessed data.
- Extract three different **feature importance** scores.
- Normalize and visualize the importance scores.

By analyzing feature importance, we can gain insights into which features are most useful for classification and potentially refine our feature selection process.

---

### 1. Splitting Data
The dataset is divided into **training** and **validation** sets using `train_test_split`. This allows us to evaluate the model’s performance on unseen data.

### 2. Training the XGBoost Model
- The dataset is converted into **DMatrix**, an optimized data structure for XGBoost.
- Model parameters are defined, including:
  - `objective='binary:logistic'`: Suitable for binary classification.
  - `max_depth=4`: Limits tree depth to control model complexity.
  - `eta=0.1`: Learning rate to adjust step size in training.
  - `eval_metric='logloss'`: Measures the performance using **logarithmic loss**.
- The model is trained using `xgb.train()` with **early stopping** to prevent overfitting.

### 3. Calculating Feature Importance
XGBoost provides feature importance based on three metrics:
- **Gain**: The average contribution of a feature when used for splitting (refer to **Section 3** of the paper for more details).
- **Cover**: The number of samples affected when a feature is used for splitting.
- **Weight**: The frequency a feature is used in boosting trees.

### 4. Normalizing Importance Values
- The calculated importance scores are stored in a **DataFrame**.
- `MinMaxScaler` is used to **scale values between 0 and 1**, making comparisons easier.
- The features are **sorted by normalized gain**, highlighting the most influential ones.

### 5. Visualizing Feature Importance
A **bar chart** is plotted to display feature importance scores (**gain, cover, and weight**).  
- The x-axis represents **feature names**.
- The y-axis shows **normalized importance scores**.

In [16]:
# Split the data into training and validation sets.
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)
#-------------------------------------------------------------------------------------------------------------------------------

# Convert datasets into DMatrix, which is optimized for XGBoost.
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)
#--------------------------------------------------------------------------------------------------------------------------------

# Define model parameters.
params = {
    'objective': 'binary:logistic',      # Binary classification objective
    'max_depth': 4,                      # Maximum depth of boosting trees
    'eta': 0.1,                          # Learning rate (step size)
    'eval_metric': 'logloss'             # Loss function to minimize (logarithmic loss)
}
#--------------------------------------------------------------------------------------------------------------------------------

# Define watchlist.
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]


num_rounds = 50         # Number of boosting rounds.
evals_result = {}       # Dictionary to store evaluation results during training

# Train the model
model_xgb = xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds=10, evals_result=evals_result, verbose_eval=True)

#-------------------------------------------------------------------------------------------------------------------------------
# Define the types of feature importance to calculate: 
# 'gain' represents the average gain (improvement) brought by a feature in the model,
# 'cover' represents the relative frequency a feature is used to split the data,
# 'weight' represents the number of times a feature is used in trees.

# Generate importance scores for each feature using the above types
importance_types = ['gain', 'cover', 'weight']
importance_values = {t: model_xgb.get_score(importance_type=t) for t in importance_types}
#-------------------------------------------------------------------------------------------------------------------------------

# Convert to DataFrame.
importance_df = pd.DataFrame(importance_values).fillna(0)

# Normalize each column to [0,1] range using MinMaxScaler.
scaler = MinMaxScaler()
importance_df[:] = scaler.fit_transform(importance_df)

# Sort by normalized gain for better visualization
importance_df = importance_df.sort_values(by='gain', ascending=False)
#--------------------------------------------------------------------------------------------------------------------------------

# Plot Feature Importance 
importance_df.plot(kind='bar', figsize=(12, 6), width=0.8, colormap='viridis')
#plt.xlabel('Features',fontsize=14)
plt.ylabel('Normalized Importance Score',fontsize=12)
# plt.title('Feature Importance (Normalized Gain, Cover, Weight)')
plt.legend(title='Metric')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
#-------------------------------------------------------------------------------------------------------------------------------

# Plot Gain Scores
plt.figure(figsize=(12, 6))
importance_df['gain'].plot(kind='bar', color='royalblue', width=0.8)

#plt.xlabel('Features', fontsize=14)
plt.ylabel('Normalized Weight Score', fontsize=12)
# plt.title('Feature Importance (Gain Only)', fontsize=14)
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


[0]	train-logloss:0.60287	eval-logloss:0.60300
[1]	train-logloss:0.52910	eval-logloss:0.52918
[2]	train-logloss:0.46758	eval-logloss:0.46775
[3]	train-logloss:0.41540	eval-logloss:0.41554
[4]	train-logloss:0.37097	eval-logloss:0.37109
[5]	train-logloss:0.33274	eval-logloss:0.33291
[6]	train-logloss:0.29958	eval-logloss:0.29973
[7]	train-logloss:0.26958	eval-logloss:0.26982
[8]	train-logloss:0.24318	eval-logloss:0.24348
[9]	train-logloss:0.22006	eval-logloss:0.22044
[10]	train-logloss:0.19988	eval-logloss:0.20027
[11]	train-logloss:0.18223	eval-logloss:0.18269
[12]	train-logloss:0.16591	eval-logloss:0.16638
[13]	train-logloss:0.15173	eval-logloss:0.15224
[14]	train-logloss:0.13905	eval-logloss:0.13960
[15]	train-logloss:0.12759	eval-logloss:0.12816
[16]	train-logloss:0.11730	eval-logloss:0.11788
[17]	train-logloss:0.10839	eval-logloss:0.10902
[18]	train-logloss:0.10046	eval-logloss:0.10113
[19]	train-logloss:0.09222	eval-logloss:0.09293
[20]	train-logloss:0.08469	eval-logloss:0.08543
[2

In [17]:
# Normalized Importance Scores for most important features 
importance_df

,gain,cover,weight
Bwd Packet Length Std,1.000000,0.619524,0.400000
Packet Length Mean,0.874347,0.861828,0.191304
Bwd Packets/s,0.186786,0.204659,0.191304
Destination Port,0.099629,0.127199,1.000000
Flow IAT Mean,0.092572,0.133998,0.078261
URG Flag Count,0.067601,0.112332,0.000000
Active Std,0.061287,0.886808,0.086957
Init_Win_bytes_backward,0.060260,0.212840,0.504348
Fwd Header Length,0.054233,0.115591,0.078261
Bwd IAT Std,0.054226,1.000000,0.043478


In [18]:
importance_df.shape

(40, 3)

# Calculating the performance metrics for XGBoost

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Make predictions on the validation set
y_pred_proba = model_xgb.predict(dvalid)  # Get probability predictions
y_pred = (y_pred_proba >= 0.5).astype(int)  # Convert probabilities to binary labels (0 or 1)

# Compute performance metrics
accuracy = accuracy_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred)
recall = recall_score(y_valid, y_pred)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')


Accuracy: 0.9960
Precision: 0.9921
Recall: 0.9970


# Compression to Decision Trees

To improve interpretability while maintaining performance, the proposed method compresses an **XGBoost model** into a **simplified decision tree**. 

### Key Steps:
- **Feature Selection**: Gain scores are used to select **the most discriminative features**.
- **Model Transformation**: Instead of using the complex ensemble of boosting trees, a decision tree is trained on the selected features.
- **Improved Interpretability**: This transformation makes predictions easier to understand without significantly compromising accuracy.


In [20]:
index_list = importance_df.index.tolist()

In [21]:
len(index_list)

40

In [22]:
# Sample the dataset randomly
N = 40000
SampledDataset = cleaned_data_labels.sample(n=N,random_state = 45)

In [23]:
# Define features and target variable
X = SampledDataset.iloc[:, :-1]  # All columns except the last one as features
y = SampledDataset.iloc[:, -1]   # Last column as the target variable

## Feature Selection and Decision Tree Training

This section evaluates how model performance varies as different numbers of important features are used.

### Steps:
1. **Data Splitting**:  
   - The dataset is divided into **training (80%)** and **testing (20%)** sets.

2. **Iterative Feature Selection & Model Training**:  
   - A loop iterates through different number of feature.
   - The **Decision Tree Classifier** (with `max_depth=depth`) is trained on each subset of reduced features and tested on the corresponding test set.
   - `depth` could be defined by the user (3, 4, and 5 were considered in the paper).

3. **Model Evaluation**:  
   - After training, the model predicts labels for the test set.
   - **Precision, recall, and accuracy** are calculated for each iteration.

4. **Results Storage & Model Saving**:  
   - Performance metrics are stored in lists.
   - Each trained model is saved using `joblib.dump()` for future use.

5. **DataFrame for Comparison**:  
   - The results are converted into a DataFrame, making it easier to compare model performance across different numbers of features.

In [24]:
# Import necessary libraries
from sklearn.tree import DecisionTreeClassifier   # Decision Tree model
import joblib                                     # For saving the trained models

num_of_important_features = len(index_list)       # Get the number of important features from the provided index list

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize lists to store evaluation metrics and results
results = []
precision_list = []
recall_list = []
accuracy_list = []
#-------------------------------------------------------------------------------------------------------------------------------
# Iterate through different numbers of important features
for feature_num in range(1,num_of_important_features+1):
    # Select a subset of features to include in training and testing
    columns_to_keep = index_list[0:feature_num]
    X_train_subset = X_train.loc[:,columns_to_keep]
    X_test_subset = X_test.loc[:,columns_to_keep]
    
    # Initialize and train a Decision Tree classifier with a max depth of 4
    model = DecisionTreeClassifier(random_state=42,max_depth=4)
    model.fit(X_train_subset, y_train)
    
    # # Make predictions on the test set
    y_pred = model.predict(X_test_subset)
    
    # Calculate evaluation metrics: precision, recall, and accuracy
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store computed metrics in respective lists
    precision_list.append(precision)
    recall_list.append(recall)
    accuracy_list.append(accuracy)
    
    
   # Store the results for this feature subset
    results.append((feature_num, precision, recall, accuracy))
   # Save the trained model for this feature subset
    joblib.dump(model, f"decision_tree_{feature_num}.joblib")
#-------------------------------------------------------------------------------------------------------------------------------
    
# Convert results list into a DataFrame for better readability and analysis
results_df = pd.DataFrame(results, columns=['Number of Features', 'Precision', 'Recall', 'Accuracy'])
    

In [25]:
results_df

,Number of Features,Precision,Recall,Accuracy
0,1,0.996230,0.638812,0.868375
1,2,0.850935,0.973757,0.928750
2,3,0.945228,0.971340,0.969250
3,4,0.990886,0.938536,0.974625
4,5,0.981488,0.952003,0.976125
5,6,0.981488,0.952003,0.976125
6,7,0.991292,0.943370,0.976500
7,8,0.984380,0.979282,0.986875
8,9,0.984380,0.979282,0.986875
9,10,0.984440,0.983080,0.988250


In [26]:
# Performance metric curves
plt.plot(range(1, len(precision_list) + 1), precision_list, marker='o', label='Precision')
plt.plot(range(1, len(recall_list) + 1), recall_list, marker='s', label='Recall')
plt.plot(range(1, len(accuracy_list) + 1), accuracy_list, marker='^', label='Accuracy')

plt.xlabel('Number of features',fontsize=14)
plt.ylabel('Performance metrics',fontsize=14)
# plt.title('Model performance metrics vs. number of features')
plt.legend(fontsize=14)
plt.grid(True)
plt.show()

## Decision Tree Visualization

### Overview
To better understand the structure of the trained **Decision Tree Classifier**, we visualize it using **`graphviz`**.

### Steps:
1. **Select a Model**:  
   - Choose a trained decision tree based on the number of features used.
   - The model is loaded using `joblib.load()`.

2. **Retrieve Feature Names**:  
   - The corresponding feature names are extracted from `index_list` for proper labeling in the visualization.

3. **Export the Tree**:  
   - The `export_graphviz()` function converts the tree structure into a **DOT format** string.
   - The visualization includes:
     - Feature names at each decision node.
     - Class labels (`Class 0` for BENIGN and `Class 1` for anomalies).
     

4. **Render and Display**:  
   - The decision tree is rendered using `graphviz.Source()`.
   - The tree is **saved** as a file (e.g., **PDF** format).
   - The `graph.view()` command opens the saved visualization.

### Purpose:
This visualization helps in **interpreting the decision-making process** of the model. By examining feature splits and their thresholds, we can gain insights into how the model differentiates between classes.


In [27]:
# Import necessary libraries for visualizing the decision tree
from sklearn.tree import export_graphviz
import graphviz
import joblib

# Select the number of features to visualize the relevent decision tree
feature_num = 8  # Change this value to visualize a model trained with a different number of features

# Load the trained Decision Tree model corresponding to the selected feature subset
model = joblib.load(f"decision_tree_{feature_num}.joblib")

# Get the corresponding feature names
columns_to_keep = index_list[:feature_num]

# Export tree as a DOT format string
dot_data = export_graphviz(model, out_file=None, 
                           feature_names=columns_to_keep,  
                           class_names=['Class 0', 'Class 1'],  
                           filled=True, rounded=True, special_characters=True)  

# Render and display the tree
graph = graphviz.Source(dot_data)  
graph.render(f"decision_tree_{feature_num}")  # Saves as 'decision_tree_7.pdf' or other formats
graph.view()  # Opens the saved file


'decision_tree_8.pdf'

# Comparing with SHAP values

In [28]:
import shap

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Computing SHAP values

In [29]:
# Explain the model's predictions using SHAP
explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(X_valid)


# SHAP Summary Plot

By default, shap.summary_plot() only shows the top 20 features with the highest average absolute SHAP values. If you want to display all features, you just need to adjust the max_display parameter. Here's how you can modify your code:

``shap.summary_plot(shap_values, X_valid, show=False, max_display=X_valid.shape[1])``


## ✅  Y-Axis: Features

Each row is a feature from the dataset, ordered top-to-bottom by importance (most impactful at the top).

Importance is based on the **mean absolute SHAP value** — in other words, how much that feature typically pushes predictions up or down, regardless of direction.

## ✅ X-Axis: SHAP Value (Impact on Model Output)
This shows how much each feature value contributes to increasing or decreasing the prediction.

Positive SHAP value → pushes prediction higher
Negative SHAP value → pushes prediction lower

## ✅ Color: Feature Value
The color shows the actual value of the feature for that specific data point.

Red = high value of the feature

Blue = low value

This helps answer questions like:

"Does having a high value for this feature tend to increase or decrease the model prediction?"

## ✅ Dots: Individual data points
Each dot represents a SHAP value for a single prediction (row in your dataset) for that feature.

The spread of dots shows the distribution of effects for that feature.

In [48]:
# shap.summary_plot(shap_values, X_valid, show=True)
# The following version allow you to edit xlabel, ylabel:

# Create the SHAP summary plot
shap.summary_plot(shap_values, X_valid, show=False, max_display=10)

# Customize axis titles
plt.xlabel("SHAP values' impact on model output")
# plt.ylabel(" Features")

# Show the updated plot
plt.show()



In [32]:
# Summary plot of SHAP values for validation set
shap.summary_plot(shap_values, X_valid, plot_type="bar", show=True,  max_display=10)

In [33]:
shap_df = pd.DataFrame(shap_values, columns=X_valid.columns)
mean_abs_shap = shap_df.abs().mean().sort_values(ascending=False)

In [37]:
# mean_abs_shap.plot(kind='bar', figsize=(12, 6), title='Mean Absolute SHAP Value per Feature')
mean_abs_shap.plot(kind='bar', figsize=(12, 6))

plt.ylabel('Mean |SHAP Value|')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=90)
plt.show()


In [38]:
# Step 2: Get mean absolute SHAP values per feature
# shap_df = pd.DataFrame(shap_values, columns=X_valid.columns)
mean_abs_shap = shap_df.abs().mean().to_frame(name='shap')  # Convert to DataFrame with column name

# Step 3: Normalize SHAP values for fair comparison
mean_abs_shap['shap'] = MinMaxScaler().fit_transform(mean_abs_shap[['shap']])

# Step 4: Merge with your normalized gain/cover/weight table
combined_df = importance_df.copy()
combined_df = combined_df.merge(mean_abs_shap, left_index=True, right_index=True)

# Step 5: Sort (optional: by 'gain' or 'shap' or whatever you prefer)
combined_df = combined_df.sort_values(by='gain', ascending=False)

# Step 6: Show the comparison table
print("Feature Importance Comparison (Normalized Scores):")
print(combined_df.round(3))

# Step 7: Plot bar chart for visual comparison
combined_df.plot(kind='bar', figsize=(14, 6), width=0.8, colormap='Set2')
plt.ylabel('Normalized Importance Score', fontsize=12)
plt.title('Feature Importance Comparison (Gain, Cover, Weight, SHAP)', fontsize=14)
plt.legend(title='Metric')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


Feature Importance Comparison (Normalized Scores):
                               gain  cover  weight   shap
 Bwd Packet Length Std        1.000  0.620   0.400  1.000
 Packet Length Mean           0.874  0.862   0.191  0.521
 Bwd Packets/s                0.187  0.205   0.191  0.255
 Destination Port             0.100  0.127   1.000  0.887
 Flow IAT Mean                0.093  0.134   0.078  0.035
 URG Flag Count               0.068  0.112   0.000  0.004
 Active Std                   0.061  0.887   0.087  0.010
 Init_Win_bytes_backward      0.060  0.213   0.504  0.301
 Fwd Header Length            0.054  0.116   0.078  0.074
 Bwd IAT Std                  0.054  1.000   0.043  0.004
 Min Packet Length            0.049  0.117   0.087  0.225
Init_Win_bytes_forward        0.038  0.050   0.191  0.042
 Total Length of Bwd Packets  0.036  0.054   0.096  0.024
 Flow Packets/s               0.035  0.296   0.104  0.076
 Fwd IAT Min                  0.031  0.048   0.087  0.053
 Fwd Packet Length St

# Reduction to Decision Trees

In [40]:
# Sort features by SHAP values (descending order)
shap_sorted_features = mean_abs_shap.sort_values(by='shap', ascending=False).index.tolist()
# num_of_important_features = len(shap_sorted_features)########################

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SHAP-based result containers
results_shap = []
precision_shap = []
recall_shap = []
accuracy_shap = []

# Train Decision Trees using top-N SHAP features
for feature_num in range(1, num_of_important_features + 1):
    columns_to_keep = shap_sorted_features[:feature_num]
    X_train_subset = X_train[columns_to_keep]
    X_test_subset = X_test[columns_to_keep]
    
    model = DecisionTreeClassifier(random_state=42, max_depth=3)
    model.fit(X_train_subset, y_train)
    
    y_pred = model.predict(X_test_subset)
    
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    precision_shap.append(precision)
    recall_shap.append(recall)
    accuracy_shap.append(accuracy)
    
    results_shap.append((feature_num, precision, recall, accuracy))
    joblib.dump(model, f"shap_decision_tree_{feature_num}.joblib")

# Results DataFrame for SHAP
results_df_shap = pd.DataFrame(results_shap, columns=['Number of Features', 'Precision', 'Recall', 'Accuracy'])

# Plot performance metrics (optional if using combined plot later)
plt.plot(range(1, len(precision_shap) + 1), precision_shap, marker='o', label='Precision')
plt.plot(range(1, len(recall_shap) + 1), recall_shap, marker='s', label='Recall')
plt.plot(range(1, len(accuracy_shap) + 1), accuracy_shap, marker='^', label='Accuracy')

plt.xlabel('Number of Features (SHAP)', fontsize=14)
plt.ylabel('Performance Metrics', fontsize=14)
plt.legend(fontsize=14)
plt.grid(True)
plt.tight_layout()
plt.show()


In [41]:
results_df

,Number of Features,Precision,Recall,Accuracy
0,1,0.996230,0.638812,0.868375
1,2,0.850935,0.973757,0.928750
2,3,0.945228,0.971340,0.969250
3,4,0.990886,0.938536,0.974625
4,5,0.981488,0.952003,0.976125
5,6,0.981488,0.952003,0.976125
6,7,0.991292,0.943370,0.976500
7,8,0.984380,0.979282,0.986875
8,9,0.984380,0.979282,0.986875
9,10,0.984440,0.983080,0.988250


In [42]:

# Choose number of features (e.g., top 8 SHAP features)
feature_num = 8
columns_to_keep = shap_sorted_features[:feature_num]

model = joblib.load(f"shap_decision_tree_{feature_num}.joblib")

dot_data = export_graphviz(model, out_file=None, 
                           feature_names=columns_to_keep,
                           class_names=['Class 0', 'Class 1'],
                           filled=True, rounded=True, special_characters=True)

graph = graphviz.Source(dot_data)
graph.render(f"shap_decision_tree_{feature_num}")
graph.view()


'shap_decision_tree_8.pdf'

# Selecting features by Cover scores 



In [51]:
# Sort features by normalized Cover scores in descending order
cover_sorted_features = combined_df.sort_values(by='cover', ascending=False).index.tolist()

# Initialize containers
results_cover = []
precision_cover = []
recall_cover = []
accuracy_cover = []

# Loop over top-N features based on Cover scores
for feature_num in range(1, len(cover_sorted_features) + 1):
    columns_to_keep = cover_sorted_features[:feature_num]
    X_train_subset = X_train[columns_to_keep]
    X_test_subset = X_test[columns_to_keep]
    
    model = DecisionTreeClassifier(random_state=42, max_depth=4)
    model.fit(X_train_subset, y_train)
    
    y_pred = model.predict(X_test_subset)
    
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    precision_cover.append(precision)
    recall_cover.append(recall)
    accuracy_cover.append(accuracy)
    
    results_cover.append((feature_num, precision, recall, accuracy))
    joblib.dump(model, f"cover_decision_tree_{feature_num}.joblib")

# Convert to DataFrame
results_df_cover = pd.DataFrame(results_cover, columns=['Number of Features', 'Precision', 'Recall', 'Accuracy'])


In [52]:
results_df_cover

,Number of Features,Precision,Recall,Accuracy
0,1,0.788732,0.599448,0.796875
1,2,0.797410,0.595304,0.798750
2,3,0.811961,0.975138,0.909250
3,4,0.833090,0.984116,0.922875
4,5,0.832360,0.984116,0.922500
5,6,0.833090,0.984116,0.922875
6,7,0.833821,0.984116,0.923250
7,8,0.888572,0.969268,0.944875
8,9,0.925024,0.984116,0.965375
9,10,0.952100,0.947169,0.963625


# Selecting features by Weight scores

In [53]:
# Sort features by normalized Weight scores in descending order
weight_sorted_features = combined_df.sort_values(by='weight', ascending=False).index.tolist()

# Initialize containers
results_weight = []
precision_weight = []
recall_weight = []
accuracy_weight = []

# Loop over top-N features based on Weight scores
for feature_num in range(1, len(weight_sorted_features) + 1):
    columns_to_keep = weight_sorted_features[:feature_num]
    X_train_subset = X_train[columns_to_keep]
    X_test_subset = X_test[columns_to_keep]
    
    model = DecisionTreeClassifier(random_state=42, max_depth=4)
    model.fit(X_train_subset, y_train)
    
    y_pred = model.predict(X_test_subset)
    
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    precision_weight.append(precision)
    recall_weight.append(recall)
    accuracy_weight.append(accuracy)
    
    results_weight.append((feature_num, precision, recall, accuracy))
    joblib.dump(model, f"weight_decision_tree_{feature_num}.joblib")

# Convert to DataFrame
results_df_weight = pd.DataFrame(results_weight, columns=['Number of Features', 'Precision', 'Recall', 'Accuracy'])


In [46]:
results_df_weight

,Number of Features,Precision,Recall,Accuracy
0,1,0.841860,1.000000,0.932000
1,2,0.920599,0.996892,0.967750
2,3,0.920675,0.997928,0.968125
3,4,0.952223,0.991022,0.978750
4,5,0.935348,0.994130,0.973000
5,6,0.983961,0.974448,0.985000
6,7,0.983961,0.974448,0.985000
7,8,0.983961,0.974448,0.985000
8,9,0.983961,0.974448,0.985000
9,10,0.982651,0.977901,0.985750


# Comparing the performance of decision trees

In [54]:
import matplotlib.pyplot as plt

# Number of features (X-axis)
# x_range = range(1, len(precision_list) + 1)

x_range = range(1, 41)

# Plot Precision
plt.figure(figsize=(10, 6))
plt.plot(x_range, precision_list, label='Gain', marker='o')
plt.plot(x_range, precision_shap, label='SHAP', marker='s')
# plt.plot(x_range, precision_cover, label='Cover', marker='^')
# plt.plot(x_range, precision_weight, label='Weight', marker='D')
plt.xlabel('Number of Features', fontsize=13)
plt.ylabel('Precision', fontsize=13)
plt.title('Precision vs Number of Features', fontsize=15)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot Recall
plt.figure(figsize=(10, 6))
plt.plot(x_range, recall_list, label='Gain', marker='o')
plt.plot(x_range, recall_shap, label='SHAP', marker='s')
# plt.plot(x_range, recall_cover, label='Cover', marker='^')
# plt.plot(x_range, recall_weight, label='Weight', marker='D')
plt.xlabel('Number of Features', fontsize=13)
plt.ylabel('Recall', fontsize=13)
plt.title('Recall vs Number of Features', fontsize=15)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot Accuracy
plt.figure(figsize=(10, 6))
plt.plot(x_range, accuracy_list, label='Gain', marker='o')
plt.plot(x_range, accuracy_shap, label='SHAP', marker='s')
# plt.plot(x_range, accuracy_cover, label='Cover', marker='^')
# plt.plot(x_range, accuracy_weight, label='Weight', marker='D')
plt.xlabel('Number of Features', fontsize=13)
plt.ylabel('Accuracy', fontsize=13)
plt.title('Accuracy vs Number of Features', fontsize=15)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
